### Data Loading

In [1]:
import os
import torch

os.environ['CUDA_VISIBLE_DEVICES'] = '3'
torch.cuda.set_device(0)

import torch.nn as nn
import yaml
import glob
import scipy.io as scio
import random
import numpy as np

from mmfi_mmwave import make_dataset, make_dataloader
from tqdm import tqdm
from evaluate import error
from mmwave_point_transformer import PointTransformerReg


In [2]:
dataset_root = '../../../data/MMFi_Dataset/'
# xian zai shi yong de shi Radar_Fused
with open('config.yaml', 'r') as fd:
    config = yaml.load(fd, Loader=yaml.FullLoader)

train_dataset, val_dataset = make_dataset(dataset_root, config)

# rng_generator = torch.manual_seed(config['init_rand_seed'])
# train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'])
# val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'])

# # TODO: Code for training or validation
# sample = train_dataset[0]
# print(sample)

mmwave
S01 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S02 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
/n
mmwave
S05 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S10 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S15 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S20 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S25 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S30 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S35 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A18', 'A19', 'A20', 'A21', 'A22', 'A23', 'A27']
S40 ['A02', 'A03', 'A04', 'A05', 'A13', 'A14', 'A17', 'A

In [3]:
def generate_limb_length(kpt_batch, normalize = False):

    limb_list = []
    limb_pairs = [[0, 1], [1, 2], [2, 3],
                            [0, 4], [4, 5], [5, 6],
                            [0, 7], [7, 8], [8, 9], [9,10],
                            [8, 11], [11, 12], [12, 13],
                            [8, 14], [14, 15], [15, 16]]
    for i in range(len(limb_pairs)):
        limb_src = limb_pairs[i][0]
        limb_end = limb_pairs[i][1]
        limb_vector = kpt_batch[:, limb_src, :] - kpt_batch[:, limb_end, :]
        limb_list.append(limb_vector)
    _limb_3d_mm = np.stack(limb_list, axis = 1)
    _limb_3d_mm = np.sqrt(np.sum(np.power(_limb_3d_mm, 2), axis=-1))
    return _limb_3d_mm




In [4]:
def collate_fn_padd(batch):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
 
    # input_modalities = list(batch[0].keys())[-1]

    scene = torch.tensor(np.array([int(t["scene"][1:]) for t in batch]))
    subject = torch.tensor(np.array([int(t["subject"][1:]) for t in batch]))
    action = torch.tensor(np.array([int(t["action"][1:]) for t in batch]))
    idx = torch.tensor(np.array([int(t["idx"]) for t in batch]))
    

    
    ## get kpts
    kpts = []
    [kpts.append(np.array(t['output'])) for t in batch]
    kpts = torch.FloatTensor(np.array(kpts))
    # kpts = kpts[:,:,:] - kpts[:,:1,:]

    # ##  get rgbs
    # rgbs = []
    # for t in batch:
    #     rgbs.append(np.array(t['input_infra1']))
    # rgb = torch.FloatTensor(np.array(rgbs))


    limb = generate_limb_length(kpts)
    limb = torch.FloatTensor(limb)

    lengths_list = []
    batch_list = []
    max_len = 150
    for i in range(len(batch)):
        b = batch[i]["input_mmwave"][:, [1, 2, 0, 3, 4]]
        ## get sequence lengths
        lengths_list.append(b.shape[0])
        ## padd
        appen_len = max_len - b.shape[0]
        if appen_len > 0:
            b = np.concatenate((b, np.zeros((appen_len, b.shape[1]), dtype=np.float32)), axis=0, dtype=np.float32)
        else: print("error")
        batch_list.append(b) 
        
    lengths = torch.tensor(lengths_list)
    batch = torch.tensor(np.stack(batch_list, axis=0))
    
    # ## padd
    # batch = [torch.Tensor(t[input_modalities]) for t in batch ]
    # batch = torch.nn.utils.rnn.pad_sequence(batch)
    # compute mask
    # batch = batch.permute(1,0,2)
    # if batch.shape[1] == 0:
    #     batch = torch.zeros(batch.shape[0], 1, 5)
    mask = (batch != 0)



    return scene, subject, action, idx, batch, kpts, lengths, mask, limb

In [5]:
rng_generator = torch.manual_seed(config['init_rand_seed'])
train_loader = make_dataloader(train_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'], collate_fn = collate_fn_padd)
# train_loader = make_dataloader(train_dataset, is_training=True, generator=rng_generator, **config['loader'])
# val_loader = make_dataloader(val_dataset, is_training=False, generator=rng_generator, **config['loader'])

### creating a config

In [6]:
from configparser import ConfigParser

#Get the configparser object
config_object = ConfigParser()

#Assume we need 2 sections in the config file, let's call them USERINFO and SERVERCONFIG
config_object["Lidar"] = {
    'num_point': 1024, 
    'nblocks': 5, 
    'nneighbor': 16, 
    'num_class': 17*3, 
    'input_dim': 3
}
config_object["mmwave"] = {
    'nblocks': 5, 
    'nneighbor': 16, 
    'num_class': 17*3, 
    'input_dim': 5
}

#Write the above sections to config.ini file
with open('model_config.yaml', 'w') as conf:
    config_object.write(conf)

### model

In [7]:
model = PointTransformerReg(
    input_dim = 5,
    nblocks = 5,
    n_p = 17
)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


PointTransformerReg(
  (backbone): Backbone(
    (fc1): Sequential(
      (0): Linear(in_features=5, out_features=32, bias=True)
      (1): ReLU()
      (2): Linear(in_features=32, out_features=32, bias=True)
    )
    (transformer1): TransformerBlock(
      (fc1): Linear(in_features=32, out_features=128, bias=True)
      (fc2): Linear(in_features=128, out_features=32, bias=True)
      (fc_delta): Sequential(
        (0): Linear(in_features=3, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
      (fc_gamma): Sequential(
        (0): Linear(in_features=128, out_features=128, bias=True)
        (1): ReLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
      )
      (w_qs): Linear(in_features=128, out_features=128, bias=False)
      (w_ks): Linear(in_features=128, out_features=128, bias=False)
      (w_vs): Linear(in_features=128, out_features=128, bias=False)
    )
    (transition_downs): Modul

### training

In [8]:
def test(model, tensor_loader, criterion1, criterion2, device, save_flag=None):
    model.eval()
    test_mpjpe = 0
    test_pampjpe = 0
    test_mse = 0
    test_limb = 0

    # empty list for data saving
    joint_gt_array_train = []
    joint_pred_array_train = []
    radar_feat_array_train = []
    radar_input_array_train = [] #  to be implement

    scene_array_train = []
    subject_array_train = []
    action_array_train = []
    idx_array_train = []


    for data in tqdm(tensor_loader):
        scene, subject, action, idx, inputs, labels, _, _, limb_gt = data
        inputs[:,:,:3] = inputs[:,:,:3] - labels[:,[0],:]
        labels = labels - labels[:,[0],:]

        inputs = inputs.to(device)
        labels.to(device)
        # limb_gt = limb_gt.to(device)
        labels = labels.type(torch.FloatTensor)
        outputs, feat, limb_pr = model(inputs)
        outputs = outputs.type(torch.FloatTensor)
        outputs.to(device)
        test_mse += criterion1(outputs,labels).item() * inputs.size(0)
        # test_limb += (limb_pr - limb_gt).abs().sum(dim=-1).mean(dim=0).item() * inputs.size(0)
        

        outputs = outputs.detach().numpy()
        labels = labels.detach().numpy()


        #append array
        if save_flag == "test" or save_flag == "train":
            joint_gt_array_train.append(labels)
            joint_pred_array_train.append(outputs)
            radar_feat_array_train.append(feat.cpu().detach().numpy())
            radar_input_array_train.append(inputs.cpu().detach().numpy())
            scene_array_train.append(scene.cpu().detach().numpy())
            subject_array_train.append(subject.cpu().detach().numpy())
            action_array_train.append(action.cpu().detach().numpy())
            idx_array_train.append(idx.cpu().detach().numpy())

        
        mpjpe, pampjpe = criterion2(outputs,labels)
        test_mpjpe += mpjpe.item() * inputs.size(0)
        test_pampjpe += pampjpe.item() * inputs.size(0)
    test_mpjpe = test_mpjpe/len(tensor_loader.dataset)
    test_pampjpe = test_pampjpe/len(tensor_loader.dataset)
    test_mse = test_mse/len(tensor_loader.dataset)
    # test_limb = test_limb/len(tensor_loader.dataset)
    print("mse: {:.8f}, mpjpe: {:.8f}, pampjpe: {:.8f}".format(float(test_mse), float(test_mpjpe),float(test_pampjpe)))

    # save
    if save_flag == "test":
        print("Saving Test.")
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "joint_gt_array_test.npy"), "wb") as f:
            nparray = np.concatenate(joint_gt_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "joint_pred_array_test.npy"), "wb") as f:
            nparray = np.concatenate(joint_pred_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "radar_feat_array_test.npy"), "wb") as f:
            nparray = np.concatenate(radar_feat_array_train)
            print(nparray.shape)
            np.save(f, nparray)

        # np transform and save to npz
        with open(os.path.join("data_saver_new", "radar_array_input_test.npy"), "wb") as f:
            nparray = np.concatenate(radar_input_array_train)
            print(nparray.shape)
            np.save(f, nparray)
            
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "scene_array_test.npy"), "wb") as f:
            nparray = np.concatenate(scene_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "subject_array_test.npy"), "wb") as f:
            nparray = np.concatenate(subject_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "action_array_test.npy"), "wb") as f:
            nparray = np.concatenate(action_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver_new", "idx_array_test.npy"), "wb") as f:
            nparray = np.concatenate(idx_array_train)
            print(nparray.shape)
            np.save(f, nparray)

    if save_flag == "train":
        print("Saving Test.")
        # np transform and save to npz
        with open(os.path.join("data_saver", "joint_gt_array_train.npy"), "wb") as f:
            nparray = np.concatenate(joint_gt_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver", "joint_pred_array_train.npy"), "wb") as f:
            nparray = np.concatenate(joint_pred_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver", "radar_feat_array_train.npy"), "wb") as f:
            nparray = np.concatenate(radar_feat_array_train)
            print(nparray.shape)
            np.save(f, nparray)

        # np transform and save to npz
        with open(os.path.join("data_saver", "scene_array_train.npy"), "wb") as f:
            nparray = np.concatenate(scene_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver", "subject_array_train.npy"), "wb") as f:
            nparray = np.concatenate(subject_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver", "action_array_train.npy"), "wb") as f:
            nparray = np.concatenate(action_array_train)
            print(nparray.shape)
            np.save(f, nparray)
        # np transform and save to npz
        with open(os.path.join("data_saver", "idx_array_train.npy"), "wb") as f:
            nparray = np.concatenate(idx_array_train)
            print(nparray.shape)
            np.save(f, nparray)
    
    
    return test_mpjpe

In [13]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load('./pre-train_weights/mmwave_scene(2).pt'))
# model.load_state_dict(torch.load('./pre-train_weights/mmwave_action(2).pt'))
model.to(device)
test(
    model=model,
    tensor_loader= val_loader,
    criterion1 = train_criterion,
    criterion2 = test_criterion,
    device=device,
    save_flag=""
        )

100%|██████████| 1040/1040 [00:31<00:00, 32.78it/s]


mse: 0.01539020, mpjpe: 0.16336294, pampjpe: 0.10876031
Saving Test.


FileNotFoundError: [Errno 2] No such file or directory: 'data_saver_new/joint_gt_array_test.npy'

In [10]:
def train(model, train_loader, test_loader, num_epochs, learning_rate, train_criterion, test_criterion, device):
    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)
    # optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
    # scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer,milestones=[20,40],gamma=0.1)
    # parameter_dir = './pre-train_weights/mmwave_p2_cross_subject(2).pt'
    # parameter_dir = './pre-train_weights/mmwave_action(2).pt'
    parameter_dir = './pre-train_weights/mmwave_scene(2).pt'
    best_test_mpjpe = 100
    for epoch in range(num_epochs):
        model.train()
        epoch_loss = 0
        limb_loss_train = 0
        epoch_accuracy = 0
        for data in tqdm(train_loader):
            scene, subject, action, idx, inputs, labels, _, _, limb_gt = data
            inputs[:,:,:3] = inputs[:,:,:3] - labels[:,[0],:]
            labels = labels - labels[:,[0],:]

            inputs = inputs.to(device)
            labels = labels.to(device)
            labels = labels.type(torch.FloatTensor)

            

            # limb_gt = limb_gt.to(device)
            
            optimizer.zero_grad()
            outputs, _, _ = model(inputs)
            outputs = outputs.to(device)
            outputs = outputs.type(torch.FloatTensor)
            loss = train_criterion(outputs,labels)
            # limb_loss = (limb_pr - limb_gt).abs().sum(dim=-1).mean(dim=0)

            loss = loss * 1 #+ limb_loss * 0
            loss.backward()
            # print(length)
            # print("loss is ", loss.item())
            optimizer.step()
            
            epoch_loss += loss.item() * inputs.size(0)
            # limb_loss_train += limb_loss.item() * inputs.size(0)
            # print("epoch loss is ", epoch_loss)
        epoch_loss = epoch_loss/len(train_loader.dataset)
        # limb_loss_train = limb_loss_train/len(train_loader.dataset)
        print('Epoch: {}, Loss: {:.8f}'.format(epoch, epoch_loss))
        # print('Epoch: {}, Train Limb Loss: {:.8f}'.format(epoch, limb_loss_train))
        if (epoch+1) % 1 == 0:
            test_mpjpe = test(
                model=model,
                tensor_loader=test_loader,
                criterion1 = train_criterion,
                criterion2 = test_criterion,
                device= device
            )
            if test_mpjpe <= best_test_mpjpe:
                print(f"best test mpjpe is:{test_mpjpe}")
                best_test_mpjpe = test_mpjpe
                torch.save(model.state_dict(), parameter_dir)
        # scheduler.step()
    return

In [12]:
train_criterion = nn.MSELoss()
test_criterion = error
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model.load_state_dict(torch.load('./pre-train_weights/mmwave_scene(2).pt'))
# for param in model.parameters():
#     param.requires_grad = True
# for param in model.fc4.parameters():
#     param.requires_grad = False
model.to(device)
train(
    model=model,
    train_loader= train_loader,
    test_loader= val_loader,    
    num_epochs= 10,
    learning_rate=5e-5,
    train_criterion = train_criterion,
    test_criterion = test_criterion,
    device=device 
        )

100%|██████████| 260/260 [00:21<00:00, 12.28it/s]


Epoch: 0, Loss: 0.01149440


100%|██████████| 1040/1040 [00:31<00:00, 32.95it/s]


mse: 0.01669394, mpjpe: 0.17172904, pampjpe: 0.11261334
best test mpjpe is:0.17172904292900096


100%|██████████| 260/260 [00:21<00:00, 12.10it/s]


...

mse: 0.02307250, mpjpe: 0.21834164, pampjpe: 0.13154855


100%|██████████| 260/260 [00:21<00:00, 11.89it/s]


Epoch: 9, Loss: 0.00703663


100%|██████████| 1040/1040 [00:32<00:00, 32.50it/s]

mse: 0.02069483, mpjpe: 0.20096472, pampjpe: 0.12682971
